Поставленную задачу будем интерпретировать как задачу регрессии, так как в данных имелись дробные оценки. Среди использованных признаков лучше себя на этапе экспериментирования показали сведения из комментариев пользователя, поэтому будем использовать их, представляя каждое слово в виде вещественного вектора определённой размерности.

Для решения используем нейронные сети — свёрточные и рекуррентные, так как они обычно показывают себя лучше в задаче обработки естественного языка. Обе архитектуры показывают достаточно хорошие результаты, но в силу того, что обучатся они всё же достаточно по-разному и на по-разному предобработанных данных, вместе они показывают результат лучше каждой в отдельности. Мы также экспериментировали с использованием более традиционных методов типа градиентного бустинга, но он давал результаты заметно хуже полученных из нейросетей.

Сами архитектуры не очень сложные, но мы используем множественные входы для того чтобы лучше учитывать разницу между тем, что обычно пишут в трёх различных полях. Итоговая активация — sigmoid — принимает значения в диапазоне (0, 1), так что предсказания сети переводятся в реальные как (x * 4) + 1. Это было сделано для того, чтобы предсказанный рейтинг товара всегда находился в допустимом диапазоне.

In [1]:
import re
import pandas as pd
import numpy as np
from pymystem3 import Mystem
import tensorflow as tf
from keras.models import Model
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout, Embedding, Input, Concatenate,LSTM,\
                            Bidirectional, AlphaDropout, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

np.random.seed(42)
tf.set_random_seed(42)

Using TensorFlow backend.


In [2]:
def read_and_split(filename):
    df=pd.read_csv(filename).fillna("")
    train,test=train_test_split(df,test_size=0.2)
    return train,test

def clean(x):
    return ''.join(m.lemmatize(re.sub('([^а-яa-z]+)',' ',x.lower()))).strip()

def prepare_for_rnn(train,test):
    train.comment=train.comment.apply(clean)
    train.commentNegative=train.commentNegative.apply(clean)
    train.commentPositive=train.commentPositive.apply(clean)

    test.comment=test.comment.apply(clean)
    test.commentNegative=test.comment.apply(clean)
    test.commentPositive=test.comment.apply(clean)
    
    tkn=Tokenizer(filters="")
    tkn.fit_on_texts(train.comment+train.commentNegative+train.commentPositive)
    
    comments=tkn.texts_to_sequences(train.comment)
    comments_neg=tkn.texts_to_sequences(train.commentNegative)
    comments_pos=tkn.texts_to_sequences(train.commentPositive)

    t_comments=tkn.texts_to_sequences(test.comment)
    t_comments_neg=tkn.texts_to_sequences(test.commentNegative)
    t_comments_pos=tkn.texts_to_sequences(test.commentPositive)

    c_len=int(np.percentile(list(map(len,comments)),95))
    cneg_len=int(np.percentile(list(map(len,comments_neg)),95))
    cpos_len=int(np.percentile(list(map(len,comments_pos)),95))

    c_pad=pad_sequences(comments,c_len)
    cneg_pad=pad_sequences(comments_neg,cneg_len)
    cpos_pad=pad_sequences(comments_pos,cpos_len)

    t_c_pad=pad_sequences(t_comments,c_len)
    t_cneg_pad=pad_sequences(t_comments_neg,cneg_len)
    t_cpos_pad=pad_sequences(t_comments_pos,cpos_len)

    y=((train.reting.values.astype(np.float32))-1)/4

    return c_pad,cneg_pad,cpos_pad,t_c_pad,t_cneg_pad,t_cpos_pad,y

def prepare_for_cnn(train,test):
    words = 256
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train.comment+train.commentNegative+train.commentPositive)
    sequences = tokenizer.texts_to_sequences(train.comment)
    word_index = tokenizer.word_index
    X_train = pad_sequences(sequences, maxlen = words)

    pos_sequences = tokenizer.texts_to_sequences(train.commentPositive)
    pX_train = pad_sequences(pos_sequences, maxlen = words)

    neg_sequences = tokenizer.texts_to_sequences(train.commentNegative)
    nX_train = pad_sequences(neg_sequences, maxlen = words)

    s = tokenizer.texts_to_sequences(test.comment)
    X_test = pad_sequences(s, maxlen=words)

    s = tokenizer.texts_to_sequences(test.commentPositive)
    pX_test = pad_sequences(s, maxlen=words)

    s = tokenizer.texts_to_sequences(test.commentNegative)
    nX_test = pad_sequences(s, maxlen=words)
    
    y_train=((train.reting.values.astype(np.float32))-1)/4
    return X_train,pX_train,nX_train,X_test,pX_test,nX_test,y_train

def train_rnn(c_pad,cneg_pad,cpos_pad,y):
    n_words=23023
    comm=Input((c_pad.shape[1],))
    cneg=Input((cneg_pad.shape[1],))
    cpos=Input((cpos_pad.shape[1],))

    m_comm=Masking()(comm)
    m_cneg=Masking()(cneg)
    m_cpos=Masking()(cpos)

    enc_lstm=Bidirectional(LSTM(256,return_sequences=True,dropout=0.2))
    enc2_lstm=LSTM(256,dropout=0.2)

    emb=Embedding(n_words,64)

    comm_emb=emb(m_comm)
    cneg_emb=emb(m_cneg)
    cpos_emb=emb(m_cpos)

    comm_enc=enc_lstm(comm_emb)
    cneg_enc=enc_lstm(cneg_emb)
    cpos_enc=enc_lstm(cpos_emb)

    comm_enc2=enc2_lstm(comm_enc)
    cneg_enc2=enc2_lstm(cneg_enc)
    cpos_enc2=enc2_lstm(cpos_enc)

    conc=Concatenate()([comm_enc2,cneg_enc2,cpos_enc2])

    res=Dense(64,activation="selu")(conc)
    res=Dense(1,activation="sigmoid")(res)

    model=Model([comm,cneg,cpos],res)
    model.compile("adam","mse")
    model.fit([c_pad,cneg_pad,cpos_pad],y,batch_size=512,epochs=3)
    return model
    
def train_cnn(X_train,pX_train,nX_train,y_train):
    n_words=23109
    words = 256
    
    embedding_layer = Embedding(n_words, 50, input_length=words)

    sequence_input = Input(shape=(words, ), dtype='float32')
    embedded_sequences = embedding_layer(sequence_input)

    pos_sequence_input = Input(shape=(words, ), dtype='float32')
    pos_embedded_sequences = embedding_layer(pos_sequence_input)

    neg_sequence_input = Input(shape=(words, ), dtype='float32')
    neg_embedded_sequences = embedding_layer(neg_sequence_input)

    x = Concatenate()([embedded_sequences, pos_embedded_sequences, neg_embedded_sequences])
    x = Dropout(0.2)(x)
    x = Conv1D(256, 4, padding='same')(x)
    x = MaxPooling1D(4)(x)
    x = Conv1D(256, 4, padding='same')(x)
    x = MaxPooling1D(4)(x)
    x = Conv1D(256, 4, activation='selu', padding='same')(x)
    x = MaxPooling1D(4)(x)
    x = Conv1D(256, 5, activation='selu', padding='same')(x)
    x = Flatten()(x)
    x = Dropout(0.8)(x)
    x = Dense(64, activation='selu')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[sequence_input, pos_sequence_input, neg_sequence_input], outputs=x)
    model.compile(loss='mse', optimizer='adam')
    model.fit([X_train, pX_train, nX_train], y_train, epochs=6, batch_size=512)
    return model

In [3]:
print("Reading data:")
train,test=read_and_split("X_train.csv")
print("Preparing data for RNN:")
m=Mystem()
c_pad,cneg_pad,cpos_pad,t_c_pad,t_cneg_pad,t_cpos_pad,y=prepare_for_rnn(train,test)
print("Preparing data for CNN:")
X_train,pX_train,nX_train,X_test,pX_test,nX_test,y_train=prepare_for_cnn(train,test)

Reading data:
Preparing data for RNN:
Preparing data for CNN:


In [4]:
print("Training RNN:")
rnn=train_rnn(c_pad,cneg_pad,cpos_pad,y)
print("Training CNN:")
cnn=train_cnn(X_train,pX_train,nX_train,y_train)

Training RNN:
Epoch 1/3
12469/12469 [==============================] - 14s - loss: 0.1200    
Epoch 2/3
12469/12469 [==============================] - 12s - loss: 0.0754    
Epoch 3/3
12469/12469 [==============================] - 12s - loss: 0.0426    
Training CNN:
Epoch 1/6
12469/12469 [==============================] - 3s - loss: 0.1649     
Epoch 2/6
12469/12469 [==============================] - 2s - loss: 0.1145     
Epoch 3/6
12469/12469 [==============================] - 2s - loss: 0.0839     
Epoch 4/6
12469/12469 [==============================] - 2s - loss: 0.0521     
Epoch 5/6
12469/12469 [==============================] - 2s - loss: 0.0339     
Epoch 6/6
12469/12469 [==============================] - 2s - loss: 0.0253     


Ниже можно увидеть результат на отложенной выборке по трём основным регрессионным метрикам:

In [5]:
pred_rnn=rnn.predict([t_c_pad,t_cneg_pad,t_cpos_pad])*4+1
pred_cnn=cnn.predict([X_test,pX_test,nX_test])*4+1
final_pred=np.mean([pred_rnn,pred_cnn],axis=0)
print("Mean squared error: {}\nMean absolute error: {}\nR^2 score: {}".format(mean_squared_error(test.reting,final_pred),
                                                                          mean_absolute_error(test.reting,final_pred),
                                                                          r2_score(test.reting,final_pred)))

Mean squared error: 0.975185072960638
Mean absolute error: 0.6467620786754958
R^2 score: 0.442469823665124
